# 📘 Notebook 02 – Emotion Annotation using DeepFace

## 🎯 Objective:
This notebook uses the [DeepFace](https://github.com/serengil/deepface) library to analyze facial emotions from a small batch of processed images (demo set of 10).

We:
- Connect to GCS and load resized demo images from `/processed_demo/`
- Use `DeepFace.analyze()` to extract dominant emotion, age, and gender
- Save the annotations into a CSV file `emotion_labels_demo.csv`
- Visualize the label distribution (e.g., pie chart)

> This is a demo run to validate the annotation pipeline before scaling to the full dataset.


In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=de8395655dcea951321ee878e4102b31c3b9e3ad1b4990394ffdf2374f5d7a1d
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [2]:
import os
import io
import time
import glob
import numpy as np
import pandas as pd
from PIL import Image
from deepface import DeepFace
from google.colab import auth
from google.cloud import storage

# STEP 1: Setup GCS
auth.authenticate_user()
project_id = 'exalted-summer-454012-d2'
bucket_name = 'boothill2001-dataset'
source_prefix = 'flickr_image_dataset/processed/'
gcs_upload_prefix = 'annotations/'

client = storage.Client(project=project_id)
bucket = client.bucket(bucket_name)

# Local folder lưu CSV
os.makedirs('/content/annotations', exist_ok=True)

# STEP 2: Load danh sách ảnh
blobs = list(bucket.list_blobs(prefix=source_prefix))
blobs = [b for b in blobs if b.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"📦 Tổng số ảnh: {len(blobs)}")

# STEP 3: Cấu hình batch linh hoạt
batch_size = 1000
start_batch = 0       # 👈 chỉnh batch bạn muốn chạy lại
end_batch = 32         # không bao gồm => chỉ chạy batch 1

total_images = len(blobs)
num_batches = (total_images + batch_size - 1) // batch_size
print(f"🔁 Chạy batch {start_batch} → {end_batch - 1} | Tổng batch: {num_batches}")

# STEP 4: Chạy từng batch
for batch_idx in range(start_batch, end_batch):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, total_images)
    batch = blobs[start_idx:end_idx]

    print(f"\n🚀 Đang xử lý batch {batch_idx+1}/{num_batches} | Ảnh {start_idx} → {end_idx - 1}")
    batch_annotations = []

    for i, blob in enumerate(batch):
        try:
            image_bytes = blob.download_as_bytes()
            image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            image_np = np.array(image)  # ✅ fix: chuyển PIL → numpy array

            analysis = DeepFace.analyze(
                img_path=image_np,                           # ✅ truyền mảng numpy
                actions=['emotion', 'age', 'gender', 'race'],
                enforce_detection=False                      # ✅ giữ lại để không bị crash
            )[0]

            batch_annotations.append({
                'filename': os.path.basename(blob.name),
                'age': analysis['age'],
                'gender': analysis['gender'],
                'dominant_emotion': analysis['dominant_emotion'],
                'dominant_race': analysis['dominant_race']
            })

            if (i + 1) % 50 == 0:
                print(f"  ✅ Annotated {i + 1}/{len(batch)} ảnh trong batch")

        except Exception as e:
            print(f"⚠️ Lỗi ảnh {blob.name}: {e}")
            continue

    # STEP 5: Lưu CSV local
    csv_filename = f'emotion_labels_batch_{batch_idx+1}.csv'
    local_path = f'/content/annotations/{csv_filename}'
    pd.DataFrame(batch_annotations).to_csv(local_path, index=False)
    print(f"✅ Đã lưu batch {batch_idx+1} → {csv_filename} ({len(batch_annotations)} dòng)")

    # STEP 6: Upload CSV lên GCS
    gcs_blob_path = gcs_upload_prefix + csv_filename
    blob = bucket.blob(gcs_blob_path)
    blob.upload_from_filename(local_path)
    print(f"☁️ Đã upload CSV lên GCS: {gcs_blob_path}")

    # STEP 7: Log lại
    with open('batch_log.txt', 'a') as f:
        f.write(f'✅ DONE batch {batch_idx+1}\n')

    time.sleep(2)


25-03-24 04:23:13 - Directory /root/.deepface has been created
25-03-24 04:23:13 - Directory /root/.deepface/weights has been created
📦 Tổng số ảnh: 31783
🔁 Chạy batch 23 → 30 | Tổng batch: 32

🚀 Đang xử lý batch 24/32 | Ảnh 23000 → 23999


Action: emotion:   0%|          | 0/4 [00:00<?, ?it/s]

25-03-24 04:23:53 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 70.8MB/s]
Action: age:  25%|██▌       | 1/4 [00:03<00:10,  3.64s/it]    

25-03-24 04:23:56 - age_model_weights.h5 will be downloaded...


...

  ✅ Annotated 900/1000 ảnh trong batch


Action: race: 100%|██████████| 4/4 [00:00<00:00, 36.72it/s]


  ✅ Annotated 950/1000 ảnh trong batch


Action: race: 100%|██████████| 4/4 [00:00<00:00, 35.54it/s]


  ✅ Annotated 1000/1000 ảnh trong batch
✅ Đã lưu batch 31 → emotion_labels_batch_31.csv (1000 dòng)
☁️ Đã upload CSV lên GCS: annotations/emotion_labels_batch_31.csv
